In [11]:
import os
import shutil
import supervision as sv

root_path = "D:/Downloads/street-facilities-selected"

# 데이터셋 로드
ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{root_path}/rotated-images",
    annotations_directory_path=f"{root_path}/labels-txt",
    data_yaml_path=f"{root_path}/street-facilities.yaml"
)

# 데이터셋 분리 (train 70%, val 15 % test 15%)
train_ds, val_test_ds = ds.split(split_ratio=0.7, random_state=1014, shuffle=True)
val_ds, test_ds = val_test_ds.split(split_ratio=0.5, random_state=535, shuffle=True)

print("train_ds length : ", len(train_ds))
print("val_ds length : ", len(val_ds))
print("test_ds length : ", len(test_ds))

train_ds length :  13524
val_ds length :  2898
test_ds length :  2898


Negative examples 갯수 분석

In [4]:
root_path = "D:/Downloads/street-facilities-selected"
json_path = os.path.join(root_path, "labels")

labels = [f for f in os.listdir(json_path) if f.endswith(".json")]
total_cnt = len(labels)

negative_sample_cnt = 0
for label in labels:
    state = int(label.split("_")[2]) # 0 정상 1 불량
    if state == 0:
        negative_sample_cnt += 1

print("total : ", total_cnt)
print("negative examples : ", negative_sample_cnt)

total :  8477
negative examples :  2180


### 분할된 데이터셋 클래스 분포 확인

In [9]:
import numpy as np

def count_classes(dataset : sv.DetectionDataset):
    n_classes = len(dataset.classes)
    
    count = np.zeros(n_classes, dtype=np.int16)
    for _, _, annotation in dataset.__iter__():
        # 사진 하나에 존재하는 0개 이상의 class object 들을 각각 센다
        # print(annotation) 
        for label in annotation.class_id:
            count[label] += 1

    return count

# names:
#   0: braille-block-defect
#   1: sidewalk-block-defect
#   2: bicycle-road-defect

print("class 0 (점자블록 파손부), 1(보도블럭 파손부), 2(자전거도로 파손부)")
for ds in [train_ds, val_ds, test_ds]:
    print(count_classes(ds))

class 0 (점자블록 파손부), 1(보도블럭 파손부), 2(자전거도로 파손부)
[5768 5331 7502]
[1099 1211 1572]
[1227 1130 1720]


### 혹시 모를 버그 확인  
당연하지만 빈 라벨 파일에 대해서는 세지 않는다.

In [6]:
# 정상 보도블럭에 대한 라벨 파일. (바운딩 박스 x) 단지 빈 txt 파일이다.
# root_path = "D:/Downloads/street-facilities-selected"
# empty_train_label = f"{root_path}/labels-txt/2_09_0_1_4_1_20210927_0000569294.txt" 

# list = [0, 0, 0]

# with open(empty_train_label, 'r', encoding='utf-8') as f:
#     for newline in f.readlines():
#         list[newline] += 1

# print(list)

In [10]:
# 저장할 디렉토리 경로 설정
root_path = "D:/Downloads/street-facilities-3"
train_images_dir = f"{root_path}/images/train"
train_labels_dir = f"{root_path}/labels/train"

val_images_dir = f"{root_path}/images/val"
val_labels_dir = f"{root_path}/labels/val"

test_images_dir = f"{root_path}/images/test"
test_labels_dir = f"{root_path}/labels/test"

# 디렉토리 생성 (존재하지 않는 경우에만)
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

# DetectionDataset 객체의 이미지 및 라벨을 복사하여 저장
def save_dataset(dataset, images_dir_to_save, labels_dir_to_save):
    i = 1
    total = len(dataset)
    for image_path, image, annotation in dataset.__iter__():
        if i % 200 == 1 : print(f"{i} / {total}")
        # image_path : path/image_filename.jpg or .jpeg
        image_filename = os.path.basename(image_path) # 이미지 파일 이름
        label_filename = os.path.splitext(image_filename)[0] + ".txt" # 확장자명만 변경 (.jpg or .jpeg -> .txt)

        # label_path 조합
        label_path = "\\".join(image_path.split("\\")[:-2] + ["labels-txt", label_filename])
        
        # 이미지 및 라벨을 각각 지정된 폴더로 복사
        shutil.copy(image_path, os.path.join(images_dir_to_save, image_filename))
        shutil.copy(label_path, os.path.join(labels_dir_to_save, label_filename))
        i += 1


# train 데이터셋 저장
save_dataset(train_ds, train_images_dir, train_labels_dir)
# validation 데이터셋 저장
save_dataset(val_ds, val_images_dir, val_labels_dir)
# test 데이터셋 저장
save_dataset(test_ds, test_images_dir, test_labels_dir)

print("Train/Validation/Test 데이터셋 저장 완료.")

1 / 13524


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Downloads\\street-facilities-selected\\labels-txt\\2_09_1_1_1_4_20211003_0000613150-4.txt'